In [1]:
import os
import pandas as pd

In [2]:
data_dir = os.path.join('../', 'out', 'sindy-out-processed')
eval_file = 'cumulative_results.csv'

In [3]:
df = pd.read_csv(os.path.join(data_dir, eval_file))
df.describe()

,vx_error,mean_adjustment_vx,time_shift_vx,vy_error,mean_adjustment_vy,time_shift_vy,l2_error
count,1441.000000,1441.000000,1441.000000,1441.000000,1441.000000,1441.000000,1441.000000
mean,189.699371,-0.044134,0.524895,192.597056,-0.181346,0.375378,328.652159
std,269.732576,0.794060,2.144014,277.873437,0.780429,2.513176,339.134615
min,0.011174,-2.603501,-4.481685,0.031567,-2.587219,-5.695787,0.269293
25%,21.337781,-0.563363,-0.745945,17.048379,-0.717107,-1.494720,92.898624
50%,87.625094,-0.043391,0.417426,88.159030,-0.139241,0.036898,230.645305
75%,245.639275,0.493050,1.758729,261.133623,0.331338,2.692135,448.316043
max,2405.623606,2.830252,5.361715,2703.950757,3.001654,5.535521,2708.324862
